In [ ]:
# ===============================
# 1. Upload file in Google Colab
# ===============================
from google.colab import files
import pandas as pd
import numpy as np

uploaded = files.upload()  # upload HRDataset_v14.csv

# Get uploaded filename
filename = list(uploaded.keys())[0]

df = pd.read_excel(filename)

print("Original shape:", df.shape)

Saving HRIS_data.xlsx to HRIS_data (2).xlsx
Original shape: (311, 36)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Employee_Name               311 non-null    object        
 1   EmpID                       311 non-null    int64         
 2   MarriedID                   311 non-null    int64         
 3   MaritalStatusID             311 non-null    int64         
 4   GenderID                    311 non-null    int64         
 5   EmpStatusID                 311 non-null    int64         
 6   DeptID                      311 non-null    int64         
 7   PerfScoreID                 311 non-null    int64         
 8   FromDiversityJobFairID      311 non-null    int64         
 9   Salary                      311 non-null    int64         
 10  Termd                       311 non-null    int64         
 11  PositionID                  311 non-null    int64         

In [ ]:
df["DateofHire"].nunique()


101

In [ ]:
import pandas as pd

# Date columns
date_cols= [
    "DateofHire",
    "DateofTermination",
    "LastPerformanceReview_Date"
]

dob_col = "DOB"

def parse_dates_strict(s):
    # Preserve NaT
    is_null = s.isna()

    # Work only on non-null values
    s2 = s.loc[~is_null].astype(str).str.replace("/", "-", regex=False)

    mask_mmddyyyy = s2.str.match(r"^\d{1,2}-\d{1,2}-\d{4}$")
    mask_mmddyy   = s2.str.match(r"^\d{1,2}-\d{1,2}-\d{2}$")
    mask_iso      = s2.str.match(r"^\d{4}-\d{2}-\d{2}$")

    valid = mask_mmddyyyy | mask_mmddyy | mask_iso
    if not valid.all():
        bad = s2[~valid]
        raise ValueError(f"Unexpected date format found:\n{bad.unique()}")

    out = pd.Series(pd.NaT, index=s.index, dtype="datetime64[ns]")

    out.loc[s2.index[mask_mmddyyyy]] = pd.to_datetime(
        s2[mask_mmddyyyy], format="%m-%d-%Y"
    )
    out.loc[s2.index[mask_mmddyy]] = pd.to_datetime(
        s2[mask_mmddyy], format="%m-%d-%y"
    )
    out.loc[s2.index[mask_iso]] = pd.to_datetime(
        s2[mask_iso], format="%Y-%m-%d"
    )

    return out


def fix_dob_two_digit_years(dob_series):
    """
    Fix DOB values incorrectly parsed into the future
    (e.g. 66 -> 2066 should be 1966)
    """
    mask_future = (
        dob_series.notna() &
        (dob_series > pd.Timestamp.today())
    )

    dob_series.loc[mask_future] = (
        dob_series.loc[mask_future] - pd.DateOffset(years=100)
    )

    return dob_series


# ---- APPLY LOGIC ----

# Parse general date columns
for col in date_cols:
    df[col] = parse_dates_strict(df[col])

# Parse DOB, then fix 2-digit year issue
df[dob_col] = parse_dates_strict(df[dob_col])
df[dob_col] = fix_dob_two_digit_years(df[dob_col])


/tmp/ipython-input-1112333204.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dob_series.loc[mask_future] = (


In [ ]:
# 1. No DOB in the future
df.loc[df["DOB"] > pd.Timestamp.today(), ["Employee_Name", "DOB"]]

# 2. Datatypes are correct
df[date_cols + ["DOB"]].dtypes

# 3. NaT preserved
df[date_cols + ["DOB"]].isna().sum()


,0
DateofHire,0
DateofTermination,207
LastPerformanceReview_Date,2
DOB,0


CHECKS:

In [ ]:
df[date_cols].isna().sum()

,0
DateofHire,0
DateofTermination,207
LastPerformanceReview_Date,0
DOB,0


In [ ]:
df[date_cols].applymap(lambda x: not (pd.isna(x) or isinstance(x, pd.Timestamp))).sum()

/tmp/ipython-input-2882765557.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[date_cols].applymap(lambda x: not (pd.isna(x) or isinstance(x, pd.Timestamp))).sum()


,0
DateofHire,0
DateofTermination,0
LastPerformanceReview_Date,0
DOB,0


In [ ]:
invalid_term = df.loc[
    df["DateofTermination"].notna() &
    df["DateofHire"].notna() &
    (df["DateofTermination"] < df["DateofHire"])
]

invalid_term.shape[0]

0

In [ ]:
invalid_review = df.loc[
    df["LastPerformanceReview_Date"].notna() &
    df["DateofHire"].notna() &
    (df["LastPerformanceReview_Date"] < df["DateofHire"])
]

invalid_review.shape[0]

2

We will see these scenarios and fix it as date of hire cannot be after last performance review date.

In [ ]:
future_dob = df.loc[df["DOB"] > pd.Timestamp.today()]

future_dob.shape[0]

25

We will see these scenarios and fix it as date of birth cannot be in future.

In [ ]:
df.loc[df["DOB"].dt.year < 1940, "DOB"].describe()


,DOB
count,0
mean,NaT
min,NaT
25%,NaT
50%,NaT
75%,NaT
max,NaT


In [ ]:
assert df["DateofHire"].notna().any(), "All hire dates missing"
assert not (df["DateofTermination"] < df["DateofHire"]).any(), "Termination before hire"
assert not (df["DOB"] > pd.Timestamp.today()).any(), "Future DOB found"

In [ ]:
future_dob = df.loc[df["DOB"] > pd.Timestamp.today()]

future_dob.shape[0]


0

let's solve two issues:

In [ ]:
bad_reviews = df.loc[
    df["LastPerformanceReview_Date"].notna() &
    df["DateofHire"].notna() &
    (df["LastPerformanceReview_Date"] < df["DateofHire"])
]

In [ ]:
bad_reviews[
    ["Employee_Name", "DateofHire", "LastPerformanceReview_Date"]
].sort_values("DateofHire")


,Employee_Name,DateofHire,LastPerformanceReview_Date
104,"Gill, Whitney",2014-07-07,2014-01-15
108,"Goble, Taisha",2015-02-16,2015-01-20


In [ ]:
mask = (
    df["LastPerformanceReview_Date"].notna() &
    df["DateofHire"].notna() &
    (df["LastPerformanceReview_Date"] < df["DateofHire"])
)

df.loc[mask, "LastPerformanceReview_Date"] = pd.NaT


In [ ]:
df.loc[
    df["Employee_Name"].isin(["Gill, Whitney", "Goble, Taisha"]),
    ["Employee_Name", "DateofHire", "LastPerformanceReview_Date"]
]

,Employee_Name,DateofHire,LastPerformanceReview_Date
108,"Goble, Taisha",2015-02-16,NaT


In [ ]:
df.loc[104]

,104
Employee_Name,"Gill, Whitney"
EmpID,10142
MarriedID,0
MaritalStatusID,4
GenderID,0
EmpStatusID,4
DeptID,6
PerfScoreID,3
FromDiversityJobFairID,0
Salary,59370


Now, DOB : No need to solve this as it was a data fixing issue intially where the year was written as yy and code fetched it wrong and assigned 20 as prefix that is whyy the dates were showing in future.

In [ ]:
future_dob = df.loc[df["DOB"] > pd.Timestamp.today()]

In [ ]:
output_path = "HRIS_data.xlsx"
df.to_excel(output_path, index=False)

In [ ]:
from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>